In [10]:
!pip install openai -q


In [11]:
import os
from openai import OpenAI

# ✅ Set Groq API key
os.environ["OPENAI_API_KEY"] = "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"

# ✅ Use Groq's OpenAI-compatible endpoint
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

# -----------------------------
# Conversation Memory with Summarization
# -----------------------------
class ConversationMemory:
    def __init__(self, max_messages=5, model="llama-3.1-8b-instant"):
        self.history = []
        self.max_messages = max_messages
        self.model = model

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        if len(self.history) > self.max_messages:
            self.summarize_history()

    def summarize_history(self):
        response = client.chat.completions.create(
            model=self.model,
            messages=self.history,
            temperature=0.3
        )
        summary = response.choices[0].message.content
        self.history = [
            {"role": "system", "content": f"Summary: {summary}"}
        ]

# -----------------------------
# Extraction Function (Structured JSON)
# -----------------------------
def extract_info_from_chat(chat_history):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        response_format={"type": "json_object"},  # ✅ FIXED
        messages=[
            {"role": "system", "content": "Extract the user's name and topic from this chat. Output JSON with keys 'name' and 'topic'."},
            {"role": "user", "content": str(chat_history)}
        ]
    )
    return response.choices[0].message.content

# -----------------------------
# Demo Usage
# -----------------------------
cm = ConversationMemory(max_messages=3)

# Add some messages
cm.add_message('user', 'Hello there!')
cm.add_message('assistant', 'Hi! How can I help?')
cm.add_message('user', 'Can you tell me a joke?')

print("After 3 messages (summarized):\n", cm.history, "\n")

cm.add_message('assistant', 'Why did the chicken cross the road? To get to the other side!')
cm.add_message('user', 'Haha, good one!')

print("After 5 messages (with truncation):\n", cm.history, "\n")

# Sample chats for extraction
samples = [
    [
        {"role":"user","content":"Hi, I am Alex."},
        {"role":"assistant","content":"Nice to meet you, Alex. What are we discussing?"},
        {"role":"user","content":"I want to talk about machine learning."}
    ],
    [
        {"role":"user","content":"Hey, I am Priya."},
        {"role":"assistant","content":"Hello Priya, what would you like to talk about?"},
        {"role":"user","content":"Let's talk about space travel."}
    ]
]

for i, chat in enumerate(samples, start=1):
    print(f"\n--- Sample Chat {i} ---")
    extracted = extract_info_from_chat(chat)
    print("Raw Extracted JSON:\n", extracted)


After 3 messages (summarized):
 [{'role': 'user', 'content': 'Hello there!'}, {'role': 'assistant', 'content': 'Hi! How can I help?'}, {'role': 'user', 'content': 'Can you tell me a joke?'}] 

After 5 messages (with truncation):
 [{'role': 'system', 'content': "Summary:  (Classic, I know, but it's a good one.)\n\nWant to hear another one?"}, {'role': 'user', 'content': 'Haha, good one!'}] 


--- Sample Chat 1 ---
Raw Extracted JSON:
 {
   "name":"Alex",
   "topic":"machine learning"
}

--- Sample Chat 2 ---
Raw Extracted JSON:
 {
   "name": "Priya",
   "topic": "space travel"
}


In [ ]:
cm = ConversationMemory(max_messages=5)  # you can change max_messages if you want

print("Start chatting! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chat ended.")
        break

    # add user message to history
    cm.add_message("user", user_input)

    # get assistant reply from Groq
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=cm.history
    )

    assistant_reply = response.choices[0].message.content
    print("Assistant:", assistant_reply)

    # add assistant reply to history
    cm.add_message("assistant", assistant_reply)

    # Show current history (optional, for debugging)
    print("\n📜 Current Conversation History:", cm.history, "\n")
